In [1]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')

from __init__ import *
from __init__jupyterlab import *

import snmcseq_utils

In [2]:
# my clusters
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_clusterings.tsv'
df_scf = pd.read_csv(f, sep='\t', index_col=0)
df_scf['cell_id'] = df_scf.index.values
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_annotation.tsv'
df_scf_annot = pd.read_csv(f, sep='\t', index_col='cluster_id_iterative')
print(df_scf.shape)
df_scf.head()

(408885, 11)


/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4,cell_id
sample,,,,,,,,,,,
snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,snmcseq_gene_2C_M_0
snmcseq_gene_2C_M_1,-2.414973,-12.098081,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-2,4-2-1,4-2-1-1,snmcseq_gene_2C_M_1
snmcseq_gene_2C_M_100,-1.446751,-12.051894,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-1,4-1-1,4-1-1-1,snmcseq_gene_2C_M_100
snmcseq_gene_2C_M_1000,-1.901734,-13.660591,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-2,4-2-1,4-2-1-1,snmcseq_gene_2C_M_1000
snmcseq_gene_2C_M_1001,-0.981086,-11.936621,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-1,4-1-1,4-1-1-1,snmcseq_gene_2C_M_1001


In [27]:
df_all = df_scf[[
#     'modality_name', 
#     'single_modality_cluster', 
    'joint_cluster_round2', 'joint_cluster_round3'
]].copy().reset_index().rename(columns={
    'joint_cluster_round2': 'SCF_round2', 
    'joint_cluster_round3': 'SCF_round3',
    'sample': 'cell_id',
})

print(df_all.shape)
df_all.head()

(408885, 3)


,cell_id,SCF_round2,SCF_round3
0,snmcseq_gene_2C_M_0,1-1,1-1-1
1,snmcseq_gene_2C_M_1,4-2,4-2-1
2,snmcseq_gene_2C_M_100,4-1,4-1-1
3,snmcseq_gene_2C_M_1000,4-2,4-2-1
4,snmcseq_gene_2C_M_1001,4-1,4-1-1


In [3]:
print(df_scf_annot.shape)
df_scf_annot.head()

(56, 4)


,cluster_id,annotation_auto_sc10xv2,annotation_curated,color
cluster_id_iterative,,,,
1-1-1,1,L5 IT Rspo1_1,L5 IT Rspo1,#3CBC78
1-2-1,2,L5 IT Rspo2_1,L5 IT Rspo2,#3CBC45
1-2-2,3,L5 IT Tcap_1,L5 IT Tcap,#5DDB65
2-1-1,4,L6 CT Cpa6,L6 CT Cpa6 (1),#338C5E
2-1-2,5,L6 CT Cpa6,L6 CT Cpa6 (2),#338C5E


In [4]:
def remove_dataset_header(cell_name):
    """
    """
    datasets = [
        '10x_nuclei_v3_macosko', # this is important: 10x_nuclei_v3_macosko has to come first
        '10x_nuclei_v3',
        'smarter_cells', 
        'smarter_nuclei',
        '10x_cells_v2', 
        '10x_cells_v3', 
        '10x_nuclei_v2',
        'snmcseq_gene',
        'snatac_gene',
        ]
    for dataset in datasets:
        if cell_name.startswith(dataset):
            cell_name_new = cell_name[len(dataset)+1:]
            return cell_name_new
    return cell_name

In [5]:
df_scf['cell'] = df_scf['cell_id'].apply(remove_dataset_header)
print(df_scf.shape)
df_scf.head()

(408885, 12)


,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4,cell_id,cell
sample,,,,,,,,,,,,
snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,snmcseq_gene_2C_M_0,2C_M_0
snmcseq_gene_2C_M_1,-2.414973,-12.098081,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-2,4-2-1,4-2-1-1,snmcseq_gene_2C_M_1,2C_M_1
snmcseq_gene_2C_M_100,-1.446751,-12.051894,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-1,4-1-1,4-1-1-1,snmcseq_gene_2C_M_100,2C_M_100
snmcseq_gene_2C_M_1000,-1.901734,-13.660591,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-2,4-2-1,4-2-1-1,snmcseq_gene_2C_M_1000,2C_M_1000
snmcseq_gene_2C_M_1001,-0.981086,-11.936621,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-1,4-1-1,4-1-1-1,snmcseq_gene_2C_M_1001,2C_M_1001


In [8]:
# zizhen clusters
f = '/cndd/fangming/CEMBA/data/MOp_all/zizhen_integration/cluster.membership.csv'
df_rna = pd.read_csv(f, index_col=0)
f = '/cndd/fangming/CEMBA/data/MOp_all/zizhen_integration/cluster.annotation.csv'
df_rna_annot = pd.read_csv(f, index_col=0)

print(df_rna.shape, df_rna_annot.shape)
df_rna_annot.head()

(482712, 1) (116, 11)


,cluster_label,cluster_color,subclass_id,subclass_label,subclass_color,class_id,class_label,class_color,cl,cluster_size,size
cluster_id,,,,,,,,,,,
1,Lamp5 Pax6,#DDACC9,1,Lamp5,#DA808C,1,GABAergic,#F05A28,1,700,700
2,Lamp5 Egln3_1,#FF88AD,1,Lamp5,#DA808C,1,GABAergic,#F05A28,2,618,618
3,Lamp5 Egln3_2,#DD8091,1,Lamp5,#DA808C,1,GABAergic,#F05A28,3,814,814
4,Lamp5 Egln3_3,#F08E98,1,Lamp5,#DA808C,1,GABAergic,#F05A28,4,1276,1276
5,Lamp5 Pdlim5_1,#FF7290,1,Lamp5,#DA808C,1,GABAergic,#F05A28,5,1288,1288


In [14]:
df_rna['modality_allen'] = df_rna.reset_index()['index'].apply(lambda x: x.split('.')[0]).values
df_rna['cell'] = df_rna.reset_index()['index'].apply(lambda x: x.split('.')[1]).values
modality_map = {
    'SmartSeq_cells_AIBS': 'smarter_cells', 
    'SmartSeq_nuclei_AIBS': 'smarter_nuclei',
    '10X_cells_v2_AIBS': '10x_cells_v2', 
    '10X_cells_v3_AIBS': '10x_cells_v3', 
    '10X_nuclei_v3_AIBS': '10x_nuclei_v3',
    '10X_nuclei_v3_Broad': '10x_nuclei_v3_macosko',
    '10X_nuclei_v2_AIBS': '10x_nuclei_v2',
    }

df_rna['modality'] = df_rna['modality_allen'].apply(lambda x: modality_map[x])
df_rna['cell_id'] = df_rna.apply(lambda x: x.modality + '_' + x.cell, axis=1)

print(df_rna.shape)
df_rna.head()

(482712, 5)


,x,modality_allen,cell,modality,cell_id
10X_cells_v2_AIBS.AAACCTGAGAAGGACA-5L8TX_171026_01_G04,7,10X_cells_v2_AIBS,AAACCTGAGAAGGACA-5L8TX_171026_01_G04,10x_cells_v2,10x_cells_v2_AAACCTGAGAAGGACA-5L8TX_171026_01_G04
10X_cells_v2_AIBS.AAACCTGAGAATCTCC-6L8TX_171026_01_B05,61,10X_cells_v2_AIBS,AAACCTGAGAATCTCC-6L8TX_171026_01_B05,10x_cells_v2,10x_cells_v2_AAACCTGAGAATCTCC-6L8TX_171026_01_B05
10X_cells_v2_AIBS.AAACCTGAGACGACGT-2L8TX_171026_01_G03,61,10X_cells_v2_AIBS,AAACCTGAGACGACGT-2L8TX_171026_01_G03,10x_cells_v2,10x_cells_v2_AAACCTGAGACGACGT-2L8TX_171026_01_G03
10X_cells_v2_AIBS.AAACCTGAGACTAAGT-10L8TX_171026_01_A04,62,10X_cells_v2_AIBS,AAACCTGAGACTAAGT-10L8TX_171026_01_A04,10x_cells_v2,10x_cells_v2_AAACCTGAGACTAAGT-10L8TX_171026_01...
10X_cells_v2_AIBS.AAACCTGAGAGACTTA-12L8TX_171026_01_D05,7,10X_cells_v2_AIBS,AAACCTGAGAGACTTA-12L8TX_171026_01_D05,10x_cells_v2,10x_cells_v2_AAACCTGAGAGACTTA-12L8TX_171026_01...


In [28]:
df_all2 = pd.merge(df_all, df_rna[['x', 'cell_id']].astype({'x': str}), on='cell_id', how='outer').rename(columns={'x': 'RNA concensus'})
print(df_all2.shape)
df_all2.head()

(546922, 4)


,cell_id,SCF_round2,SCF_round3,RNA concensus
0,snmcseq_gene_2C_M_0,1-1,1-1-1,NaN
1,snmcseq_gene_2C_M_1,4-2,4-2-1,NaN
2,snmcseq_gene_2C_M_100,4-1,4-1-1,NaN
3,snmcseq_gene_2C_M_1000,4-2,4-2-1,NaN
4,snmcseq_gene_2C_M_1001,4-1,4-1-1,NaN


### LIGER clustering
UMAP:
- Non-neuron 87143 cells
- Neurons 408,884 cells
    - IT
    - Non-IT
    - CGE
    - MGE
    - 408,736 cells in total
    
Clusters:
- First level: 408,884 cells (Neurons only)
- Second level: 495,879 cells (Neuron + Non-Neuronal)


In [38]:
dirc = '/cndd/fangming/CEMBA/data/MOp_all/josh_welch'
with snmcseq_utils.cd(dirc):
    f = 'LIGER_non_neuron_organized_FX2.tsv'
    df_liger = pd.read_csv(f, sep='\t')
    f = 'LIGER_organized_FX2.tsv'
    df_liger2 = pd.read_csv(f, sep='\t')
    
    # 
    df_liger['cell_id'] = df_liger['modality'] + '_' + df_liger['cell']
    df_liger2['cell_id'] = df_liger2['modality'] + '_' + df_liger2['cell']
    
    # 
    df_liger['level1_cluster'] = 'glia'
    
# df_liger = pd.merge(df_liger, df_scf[['cell', 
#                                       'modality', 'modality_name', 
#                                       'single_modality_cluster', 'single_modality_annot',
#                                      ]], on='cell')
print(df_liger.shape)
print(df_liger2.shape)
df_liger.head()

(86293, 9)
(408736, 14)


,cell,UMAP_1,UMAP_2,level2_cluster,annot,color,modality,cell_id,level1_cluster
0,SM-GE64U_S119_E1-50,-6.489569,3.124291,non_neuron_0,Oligo Opalin_2,#6CA491,smarter_nuclei,smarter_nuclei_SM-GE64U_S119_E1-50,glia
1,SM-GE63Z_S046_E1-50,-6.252966,2.816719,non_neuron_0,Oligo Opalin_2,#6CA491,smarter_nuclei,smarter_nuclei_SM-GE63Z_S046_E1-50,glia
2,SM-GE63Z_S063_E1-50,-3.739112,4.154798,non_neuron_0,Oligo Opalin_2,#6CA491,smarter_nuclei,smarter_nuclei_SM-GE63Z_S063_E1-50,glia
3,SM-GE63Z_S138_E1-50,-4.678784,1.695413,non_neuron_0,Oligo Opalin_2,#6CA491,smarter_nuclei,smarter_nuclei_SM-GE63Z_S138_E1-50,glia
4,SM-GE63Z_S147_E1-50,-2.644890,3.455592,non_neuron_0,Oligo Opalin_2,#6CA491,smarter_nuclei,smarter_nuclei_SM-GE63Z_S147_E1-50,glia


In [39]:
df_liger.head()

,cell,UMAP_1,UMAP_2,level2_cluster,annot,color,modality,cell_id,level1_cluster
0,SM-GE64U_S119_E1-50,-6.489569,3.124291,non_neuron_0,Oligo Opalin_2,#6CA491,smarter_nuclei,smarter_nuclei_SM-GE64U_S119_E1-50,glia
1,SM-GE63Z_S046_E1-50,-6.252966,2.816719,non_neuron_0,Oligo Opalin_2,#6CA491,smarter_nuclei,smarter_nuclei_SM-GE63Z_S046_E1-50,glia
2,SM-GE63Z_S063_E1-50,-3.739112,4.154798,non_neuron_0,Oligo Opalin_2,#6CA491,smarter_nuclei,smarter_nuclei_SM-GE63Z_S063_E1-50,glia
3,SM-GE63Z_S138_E1-50,-4.678784,1.695413,non_neuron_0,Oligo Opalin_2,#6CA491,smarter_nuclei,smarter_nuclei_SM-GE63Z_S138_E1-50,glia
4,SM-GE63Z_S147_E1-50,-2.644890,3.455592,non_neuron_0,Oligo Opalin_2,#6CA491,smarter_nuclei,smarter_nuclei_SM-GE63Z_S147_E1-50,glia


In [40]:
df_liger2.head()

,cell,level1_UMAP_1,level1_UMAP_2,level2_UMAP_1,level2_UMAP_2,level1_cluster,level2_cluster,modality,modality_name,single_modality_cluster,single_modality_annot,annot,color,cell_id
0,SM-GE653_S113_E1-50,-3.164615,12.190200,4.718955,-2.380108,10,cge_5,smarter_nuclei,SMART nuclei,2,Lamp5 Egln3_2,Lamp5 Pax6,#DDACC9,smarter_nuclei_SM-GE653_S113_E1-50
1,SM-GE653_S117_E1-50,-3.395991,11.764448,2.380869,-2.322626,10,cge_5,smarter_nuclei,SMART nuclei,2,Lamp5 Egln3_2,Lamp5 Pax6,#DDACC9,smarter_nuclei_SM-GE653_S117_E1-50
2,SM-GE653_S118_E1-50,-2.278710,12.773043,6.417862,-1.314918,8,cge_5,smarter_nuclei,SMART nuclei,9,Vip Cbln4,Lamp5 Pax6,#DDACC9,smarter_nuclei_SM-GE653_S118_E1-50
3,SM-GE653_S120_E1-50,-2.899099,12.395375,5.662322,-2.015199,10,cge_5,smarter_nuclei,SMART nuclei,2,Lamp5 Egln3_2,Lamp5 Pax6,#DDACC9,smarter_nuclei_SM-GE653_S120_E1-50
4,SM-GE653_S122_E1-50,-3.567551,11.620143,2.055580,-3.102450,10,cge_5,smarter_nuclei,SMART nuclei,2,Lamp5 Egln3_2,Lamp5 Pax6,#DDACC9,smarter_nuclei_SM-GE653_S122_E1-50


In [41]:
df_liger_all = pd.concat([
    df_liger2[['cell_id', 'level1_cluster', 'level2_cluster']],
    df_liger[['cell_id', 'level1_cluster', 'level2_cluster']],
])
print(df_liger_all.shape)
df_liger_all.head()

(495029, 3)


,cell_id,level1_cluster,level2_cluster
0,smarter_nuclei_SM-GE653_S113_E1-50,10,cge_5
1,smarter_nuclei_SM-GE653_S117_E1-50,10,cge_5
2,smarter_nuclei_SM-GE653_S118_E1-50,8,cge_5
3,smarter_nuclei_SM-GE653_S120_E1-50,10,cge_5
4,smarter_nuclei_SM-GE653_S122_E1-50,10,cge_5


In [43]:
df_all3 = pd.merge(df_all2, df_liger_all.rename(columns={'level1_cluster': 'LIGER_level1', 
                                                         'level2_cluster': 'LIGER_level2', 
                                                        }), on='cell_id', how='outer')
print(df_all3.shape)
df_all3.head()

(571703, 6)


,cell_id,SCF_round2,SCF_round3,RNA concensus,LIGER_level1,LIGER_level2
0,snmcseq_gene_2C_M_0,1-1,1-1-1,NaN,0,it_8
1,snmcseq_gene_2C_M_1,4-2,4-2-1,NaN,1,it_0
2,snmcseq_gene_2C_M_100,4-1,4-1-1,NaN,1,it_0
3,snmcseq_gene_2C_M_1000,4-2,4-2-1,NaN,1,it_0
4,snmcseq_gene_2C_M_1001,4-1,4-1-1,NaN,1,it_0


In [45]:
def remove_dataset_header(cell_name):
    """
    """
    datasets = [
        '10x_nuclei_v3_macosko', # this is important: 10x_nuclei_v3_macosko has to come first
        '10x_nuclei_v3',
        'smarter_cells', 
        'smarter_nuclei',
        '10x_cells_v2', 
        '10x_cells_v3', 
        '10x_nuclei_v2',
        'snmcseq_gene',
        'snatac_gene',
        ]
    for dataset in datasets:
        if cell_name.startswith(dataset):
            cell_name_new = cell_name[len(dataset)+1:]
            return cell_name_new
    return cell_name

def get_dataset_header(cell_name):
    """
    """
    datasets = [
        '10x_nuclei_v3_macosko', # this is important: 10x_nuclei_v3_macosko has to come first
        '10x_nuclei_v3',
        'smarter_cells', 
        'smarter_nuclei',
        '10x_cells_v2', 
        '10x_cells_v3', 
        '10x_nuclei_v2',
        'snmcseq_gene',
        'snatac_gene',
        ]
    for dataset in datasets:
        if cell_name.startswith(dataset):
            return dataset 
    return np.nan

In [46]:
df_all3['modality'] = df_all3['cell_id'].apply(get_dataset_header)
print(df_all3.shape)
df_all3.head()

(571703, 7)


,cell_id,SCF_round2,SCF_round3,RNA concensus,LIGER_level1,LIGER_level2,modality
0,snmcseq_gene_2C_M_0,1-1,1-1-1,NaN,0,it_8,snmcseq_gene
1,snmcseq_gene_2C_M_1,4-2,4-2-1,NaN,1,it_0,snmcseq_gene
2,snmcseq_gene_2C_M_100,4-1,4-1-1,NaN,1,it_0,snmcseq_gene
3,snmcseq_gene_2C_M_1000,4-2,4-2-1,NaN,1,it_0,snmcseq_gene
4,snmcseq_gene_2C_M_1001,4-1,4-1-1,NaN,1,it_0,snmcseq_gene


In [51]:
modality_rename = {
    'snmcseq_gene': 'DNA methylation', 
    'snatac_gene': 'Open chromatin', 
    'smarter_cells': 'SmartSeq_cells_AIBS', 
    'smarter_nuclei': 'SmartSeq_nuclei_AIBS',
    '10x_cells_v2': '10X_cells_v2_AIBS', 
    '10x_cells_v3': '10X_cells_v3_AIBS', 
    '10x_nuclei_v3': '10X_nuclei_v3_AIBS',
    '10x_nuclei_v3_macosko': '10X_nuclei_v3_Broad', 
    '10x_nuclei_v2': '10X_nuclei_v2_AIBS',
}

df_all3['modality'] = df_all3['modality'].apply(lambda x: modality_rename[x])
df_all3['modality'].unique()

array(['snmcseq_gene', 'snatac_gene', 'smarter_cells', 'smarter_nuclei',
       '10x_cells_v2', '10x_cells_v3', '10x_nuclei_v3',
       '10x_nuclei_v3_macosko', '10x_nuclei_v2'], dtype=object)

In [55]:
print(df_all3.shape)
print(df_all3[~df_all3['modality'].isnull()].shape)

print(df_all3[~df_all3['SCF_round2'].isnull()].shape)
print(df_all3[~df_all3['SCF_round3'].isnull()].shape)

print(df_all3[~df_all3['RNA concensus'].isnull()].shape)

print(df_all3[~df_all3['LIGER_level1'].isnull()].shape)
print(df_all3[~df_all3['LIGER_level2'].isnull()].shape)

(571703, 7)
(571703, 7)
(408885, 7)
(408885, 7)
(482712, 7)
(495029, 7)
(495029, 7)


In [54]:
print(df_all3.shape)
df_all3.head()

(571703, 7)


,cell_id,SCF_round2,SCF_round3,RNA concensus,LIGER_level1,LIGER_level2,modality
0,snmcseq_gene_2C_M_0,1-1,1-1-1,NaN,0,it_8,DNA methylation
1,snmcseq_gene_2C_M_1,4-2,4-2-1,NaN,1,it_0,DNA methylation
2,snmcseq_gene_2C_M_100,4-1,4-1-1,NaN,1,it_0,DNA methylation
3,snmcseq_gene_2C_M_1000,4-2,4-2-1,NaN,1,it_0,DNA methylation
4,snmcseq_gene_2C_M_1001,4-1,4-1-1,NaN,1,it_0,DNA methylation


In [56]:
# f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/cell_cluster_assignment_all_integrations.tsv'
# df_all3.to_csv(f, sep='\t', header=True, index=False, na_rep='NA')

In [77]:
# all annotations

f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_annotation_round2.tsv'
annot1 = pd.read_csv(f, sep='\t')
annot1 = annot1.rename(columns={'cluster_id_iterative': 'cluster', 
                                'annotation_curated': 'annot',
                                'color': 'color',
                               })
print(annot1.shape)
annot1.head()

(29, 3)


,cluster,annot,color
0,1-1,L5 IT Rspo1,#3CBC78
1,1-2,L5 IT Rspo2_Tcap,#3CBC45
2,2-1,L6 CT Cpa6,#338C5E
3,2-2,L6 CT Cpa6_Gpr139,#338C5E
4,2-3,L6 CT Nxph2,#3E766C


In [78]:
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_annotation_round3.tsv'
annot2 = pd.read_csv(f, sep='\t')
annot2 = annot2.rename(columns={'cluster_id_iterative': 'cluster', 
                                'annotation_curated': 'annot',
                                'color': 'color',
                               })[['cluster', 'annot', 'color']]
print(annot2.shape)
annot2.head()

(56, 3)


,cluster,annot,color
0,1-1-1,L5 IT Rspo1,#3CBC78
1,1-2-1,L5 IT Rspo2,#3CBC45
2,1-2-2,L5 IT Tcap,#5DDB65
3,2-1-1,L6 CT Cpa6 (1),#338C5E
4,2-1-2,L6 CT Cpa6 (2),#338C5E


In [79]:
f = '/cndd/fangming/CEMBA/data/MOp_all/zizhen_integration/cluster.annotation.csv'
annot3 = pd.read_csv(f)
annot3 = annot3.rename(columns={'cluster_id': 'cluster', 
                                'cluster_label': 'annot',
                                'cluster_color': 'color',
                               })[['cluster', 'annot', 'color']]
print(annot3.shape)
annot3.head()

(116, 3)


,cluster,annot,color
0,1,Lamp5 Pax6,#DDACC9
1,2,Lamp5 Egln3_1,#FF88AD
2,3,Lamp5 Egln3_2,#DD8091
3,4,Lamp5 Egln3_3,#F08E98
4,5,Lamp5 Pdlim5_1,#FF7290


In [80]:
f1 = '/cndd/fangming/CEMBA/data/MOp_all/josh_welch/LIGER_non_neuron_organized_annot_FX2.tsv'
f2 = '/cndd/fangming/CEMBA/data/MOp_all/josh_welch/LIGER_organized_annot_FX2.tsv'
annot4 = pd.concat([
    pd.read_csv(f1, sep='\t'), 
    pd.read_csv(f2, sep='\t'), 
])
print(annot4.shape)
annot4.head()

(71, 3)


,cluster,annot,color
0,non_neuron_0,Oligo Opalin_2,#6CA491
1,non_neuron_1,Astro Aqp4_Slc7a10,#8D7C5D
2,non_neuron_10,VLMC_5,#535946
3,non_neuron_11,Oligo Opalin_2,#6CA491
4,non_neuron_12,VLMC_3,#535944


In [81]:
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/cluster_annotation_scf_round2.tsv'
annot1.to_csv(f, sep='\t', header=True, index=False)
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/cluster_annotation_scf_round3.tsv'
annot2.to_csv(f, sep='\t', header=True, index=False)
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/cluster_annotation_rna_consensus.tsv'
annot3.to_csv(f, sep='\t', header=True, index=False)
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/cluster_annotation_liger_level2.tsv'
annot4.to_csv(f, sep='\t', header=True, index=False)